## 0. Setup
Import the Python modules used throughout the notebook. Make sure you have already installed the packages listed in the README (pandas, numpy, mutagen, unidecode).

### Package bootstrap
Install any missing Python packages required by this workflow so the import cell succeeds even on a fresh environment.

In [36]:
import importlib
import subprocess
import sys

REQUIRED_PACKAGES = {
    "pandas": "pandas",
    "numpy": "numpy",
    "mutagen": "mutagen",
    "unidecode": "Unidecode"
}

for module_name, install_name in REQUIRED_PACKAGES.items():
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing missing dependency: {install_name}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", install_name])
print("Dependency check complete.")

Dependency check complete.


In [37]:
from __future__ import annotations
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Iterable, List, Optional

import math
import re
import shutil
import pandas as pd
import numpy as np
from mutagen import File as MutagenFile
from unidecode import unidecode

## 1. Configuration
Define key directories (relative to the repository root) and ensure the output folder for reports exists.

In [38]:
# Adjust these paths if you relocate folders.
REPO_ROOT = Path.cwd()
MUSIC_ROOT = REPO_ROOT / "Music"
SPOTIFY_PLAYLISTS = REPO_ROOT / "spotify_playlists"
SHOPPING_LIST_DIR = REPO_ROOT / "shopping_lists"
LIBRARY_INDEX_CSV = REPO_ROOT / "library_index.csv"
ADD_ROOT = REPO_ROOT / "Add"

SHOPPING_LIST_DIR.mkdir(exist_ok=True)
ADD_ROOT.mkdir(exist_ok=True)
MUSIC_ROOT.mkdir(exist_ok=True)
print(f"Repository root: {REPO_ROOT}")
print(f"Music library: {MUSIC_ROOT}")
print(f"Spotify playlist CSVs: {SPOTIFY_PLAYLISTS}")
print(f"Shopping/output directory: {SHOPPING_LIST_DIR}")
print(f"Add drop directory: {ADD_ROOT}")

Repository root: c:\Users\CJ\Documents\GitHub\Exodusify
Music library: c:\Users\CJ\Documents\GitHub\Exodusify\Music
Spotify playlist CSVs: c:\Users\CJ\Documents\GitHub\Exodusify\spotify_playlists
Shopping/output directory: c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists
Add drop directory: c:\Users\CJ\Documents\GitHub\Exodusify\Add


## 2. Helper functions
Canonicalization helpers keep matching consistent between Spotify exports and local audio metadata.

In [39]:
NON_ALNUM = re.compile(r"[^a-z0-9]+")
FEAT_PATTERN = re.compile(r"\(feat\..*?\)", re.IGNORECASE)
REMIX_PATTERN = re.compile(r"-\s*(remaster(ed)?|remix|edit|mix).*", re.IGNORECASE)
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.m4a', '.aac', '.ogg', '.wav', '.aiff'}
INVALID_PATH_CHARS = re.compile(r"[<>:\"/\\|?*]")


def canonicalize_string(value: Optional[str]) -> str:
    if not value:
        return ""
    normalized = unidecode(str(value))
    normalized = FEAT_PATTERN.sub("", normalized)
    normalized = REMIX_PATTERN.sub("", normalized)
    normalized = normalized.lower()
    normalized = NON_ALNUM.sub(" ", normalized)
    normalized = normalized.strip()
    return re.sub(r"\s+", " ", normalized)


def safe_path_component(value: Optional[str], fallback: str = "Unknown") -> str:
    candidate = str(value).strip() if value else fallback
    candidate = unidecode(candidate)
    candidate = INVALID_PATH_CHARS.sub("", candidate)
    candidate = candidate.replace('/', '-').replace('\\', '-')
    candidate = candidate.strip()
    return candidate or fallback


def primary_artist(artists_field: Optional[str]) -> str:
    if not artists_field or not isinstance(artists_field, str):
        return ""
    first = artists_field.split(';')[0]
    return first.strip()


def friendly_playlist_name(csv_path: Path) -> str:
    name = csv_path.stem.replace('_', ' ')
    return name.strip()


def duration_ms_from_audio(audio_obj) -> Optional[int]:
    if audio_obj and audio_obj.info and getattr(audio_obj.info, 'length', None):
        return int(round(audio_obj.info.length * 1000))
    return None

## 4. Scan the music library
Create or refresh an auditable `library_index.csv` capturing metadata for every audio file under `Music/`.

In [40]:
def scan_music_library(music_root: Path) -> pd.DataFrame:
    records = []
    if not music_root.exists():
        print(f"Music directory not found: {music_root}")
        return pd.DataFrame()

    for file_path in music_root.rglob('*'):
        if not file_path.is_file() or file_path.suffix.lower() not in AUDIO_EXTENSIONS:
            continue
        try:
            audio = MutagenFile(file_path)
        except Exception as exc:
            print(f"Failed to read {file_path}: {exc}")
            audio = None

        tags = getattr(audio, 'tags', None) if audio else None
        artist_tag = None
        title_tag = None
        album_tag = None

        if tags:
            artist_tag = tags.get('TPE1') or tags.get('artist')
            title_tag = tags.get('TIT2') or tags.get('title')
            album_tag = tags.get('TALB') or tags.get('album')

        artist_str = str(artist_tag.text[0]) if hasattr(artist_tag, 'text') else (artist_tag if isinstance(artist_tag, str) else None)
        title_str = str(title_tag.text[0]) if hasattr(title_tag, 'text') else (title_tag if isinstance(title_tag, str) else None)
        album_str = str(album_tag.text[0]) if hasattr(album_tag, 'text') else (album_tag if isinstance(album_tag, str) else None)

        # Fallbacks from the path structure
        if not artist_str:
            artist_str = file_path.parent.name
        if not title_str:
            title_str = file_path.stem

        records.append({
            'file_path': file_path.relative_to(music_root).as_posix(),
            'artist_raw': artist_str,
            'title_raw': title_str,
            'album_raw': album_str,
            'artist_canonical': canonicalize_string(artist_str),
            'title_canonical': canonicalize_string(title_str),
            'duration_ms': duration_ms_from_audio(audio)
        })

    df = pd.DataFrame.from_records(records)
    if not df.empty:
        df.sort_values(['artist_canonical', 'title_canonical', 'file_path'], inplace=True)
    return df

library_index = scan_music_library(MUSIC_ROOT)
print(f"Indexed {len(library_index):,} local tracks")
if not library_index.empty:
    library_index.to_csv(LIBRARY_INDEX_CSV, index=False)
    display(library_index.head())
else:
    print('Library index is empty – check MUSIC_ROOT or file extensions.')

Indexed 1,321 local tracks


,file_path,artist_raw,title_raw,album_raw,artist_canonical,title_canonical,duration_ms
2,21 Savage/i am i was/a lot.mp3,21 Savage,a lot,i am > i was,21 savage,a lot,391706
4,3OH!3/Streets Of Gold/01 Beaumont.mp3,3OH!3,Beaumont,Streets Of Gold,3oh 3,beaumont,68467
14,3OH!3/WANT (Deluxe)/04 CHOKECHAIN.mp3,3OH!3,CHOKECHAIN,WANT (Deluxe),3oh 3,chokechain,211722
12,3OH!3/WANT/COLORADOSUNRISE.mp3,3OH!3,COLORADOSUNRISE,WANT,3oh 3,coloradosunrise,202841
6,3OH!3/Streets Of Gold/04 Déjà Vu.mp3,3OH!3,Déjà Vu,Streets Of Gold,3oh 3,deja vu,184712


## 5. Load Spotify playlist exports
Combine all CSV files in `spotify_playlists/` into a single DataFrame with helpful flags.

In [41]:
def load_spotify_playlists(csv_root: Path) -> pd.DataFrame:
    rows = []
    if not csv_root.exists():
        print(f"Spotify playlist directory not found: {csv_root}")
        return pd.DataFrame()

    csv_files = sorted(csv_root.glob('*.csv'))
    if not csv_files:
        print(f"No CSV files found in {csv_root}")
        return pd.DataFrame()

    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
        except Exception as exc:
            print(f"Failed to read {csv_file}: {exc}")
            continue
        df['playlist_name'] = friendly_playlist_name(csv_file)
        df['is_liked'] = csv_file.name.lower() == 'liked_songs.csv'
        df['is_top_songs'] = csv_file.name.lower().startswith('your_top_songs_')
        rows.append(df)

    merged = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    if merged.empty:
        return merged

    merged['primary_artist'] = merged['Artist Name(s)'].apply(primary_artist)
    merged['artist_canonical'] = merged['primary_artist'].apply(canonicalize_string)
    merged['title_canonical'] = merged['Track Name'].apply(canonicalize_string)
    return merged

spotify_df = load_spotify_playlists(SPOTIFY_PLAYLISTS)
print(f"Loaded {len(spotify_df):,} Spotify rows across {spotify_df['playlist_name'].nunique() if not spotify_df.empty else 0} playlists")
if not spotify_df.empty:
    display(spotify_df.head())

Loaded 14,555 Spotify rows across 74 playlists


C:\Users\CJ\AppData\Local\Temp\ipykernel_65548\1881787435.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()


,Track URI,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Explicit,Added By,Added At,...,Liveness,Valence,Tempo,Time Signature,playlist_name,is_liked,is_top_songs,primary_artist,artist_canonical,title_canonical
0,spotify:track:5PHPENfE3RVmHGAA2A7Hfx,Sissy That Walk,Born Naked,RuPaul,2014-02-24,212693.0,1.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-14T00:18:40Z,...,0.0865,0.591,126.046,4.0,2019 Second Chance Prom,False,False,RuPaul,rupaul,sissy that walk
1,spotify:track:7jman10UPhzhtOOqZLjSsh,Cover Girl,Champion,RuPaul,2009-04-07,178626.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:33:24Z,...,0.0959,0.717,127.995,4.0,2019 Second Chance Prom,False,False,RuPaul,rupaul,cover girl
2,spotify:track:25Y5EIIljjyEhKdl23zv6j,Your Makeup Is Terrible,Anus,Alaska Thunderfuck,2015-06-23,237474.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:36:45Z,...,0.0974,0.585,128.054,4.0,2019 Second Chance Prom,False,False,Alaska Thunderfuck,alaska thunderfuck,your makeup is terrible
3,spotify:track:2M2WJ7gBlcKNxdhyfPp9zY,Best of My Love,Rejoice,The Emotions,1977-06-10,220560.0,69.0,False,allyjclark-us,2019-05-13T19:40:48Z,...,0.0904,0.970,115.443,4.0,2019 Second Chance Prom,False,False,The Emotions,the emotions,best of my love
4,spotify:track:37Q5anxoGWYdRsyeXkkNoI,Heaven Is A Place On Earth,Greatest Vol.1 - Belinda Carlisle,Belinda Carlisle,1987,246520.0,27.0,False,allyjclark-us,2019-05-13T19:40:54Z,...,0.0497,0.793,122.902,4.0,2019 Second Chance Prom,False,False,Belinda Carlisle,belinda carlisle,heaven is a place on earth


## 6. Match Spotify tracks to the local library
Left-join on canonical artist/title keys and filter by duration tolerance where available.

In [42]:
DURATION_TOLERANCE_MS = 3000

def match_tracks(spotify_df: pd.DataFrame, library_df: pd.DataFrame, duration_tolerance_ms: int = DURATION_TOLERANCE_MS) -> pd.DataFrame:
    if spotify_df.empty:
        return pd.DataFrame()
    if library_df.empty:
        result = spotify_df.copy()
        result['file_path'] = pd.NA
        result['duration_ms_local'] = pd.NA
        return result

    lib_cols = library_df.rename(columns={'duration_ms': 'duration_ms_local'})
    merged = spotify_df.merge(lib_cols, how='left', on=['artist_canonical', 'title_canonical'], suffixes=('_spotify', '_local'))

    if 'duration_ms_local' in merged.columns:
        mask = merged['duration_ms_local'].notna() & merged['Duration (ms)'].notna()
        mismatched = mask & (merged['Duration (ms)'] - merged['duration_ms_local']).abs() > duration_tolerance_ms
        merged.loc[mismatched, ['file_path', 'duration_ms_local']] = pd.NA
    return merged

matched_df = match_tracks(spotify_df, library_index)
print(f"Matched rows: {len(matched_df):,}")
if not matched_df.empty:
    have_files = matched_df['file_path'].notna().sum()
    print(f"Tracks already downloaded: {have_files:,}")
    print(f"Tracks missing locally: {len(matched_df) - have_files:,}")
    display(matched_df.head())

Matched rows: 15,005
Tracks already downloaded: 4,420
Tracks missing locally: 10,585


,Track URI,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Explicit,Added By,Added At,...,is_liked,is_top_songs,primary_artist,artist_canonical,title_canonical,file_path,artist_raw,title_raw,album_raw,duration_ms_local
0,spotify:track:5PHPENfE3RVmHGAA2A7Hfx,Sissy That Walk,Born Naked,RuPaul,2014-02-24,212693.0,1.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-14T00:18:40Z,...,False,False,RuPaul,rupaul,sissy that walk,NaN,NaN,NaN,NaN,NaN
1,spotify:track:7jman10UPhzhtOOqZLjSsh,Cover Girl,Champion,RuPaul,2009-04-07,178626.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:33:24Z,...,False,False,RuPaul,rupaul,cover girl,NaN,NaN,NaN,NaN,NaN
2,spotify:track:25Y5EIIljjyEhKdl23zv6j,Your Makeup Is Terrible,Anus,Alaska Thunderfuck,2015-06-23,237474.0,0.0,False,22gilcer6gcjla3s2qjnnhgxy,2019-05-13T19:36:45Z,...,False,False,Alaska Thunderfuck,alaska thunderfuck,your makeup is terrible,NaN,NaN,NaN,NaN,NaN
3,spotify:track:2M2WJ7gBlcKNxdhyfPp9zY,Best of My Love,Rejoice,The Emotions,1977-06-10,220560.0,69.0,False,allyjclark-us,2019-05-13T19:40:48Z,...,False,False,The Emotions,the emotions,best of my love,NaN,NaN,NaN,NaN,NaN
4,spotify:track:37Q5anxoGWYdRsyeXkkNoI,Heaven Is A Place On Earth,Greatest Vol.1 - Belinda Carlisle,Belinda Carlisle,1987,246520.0,27.0,False,allyjclark-us,2019-05-13T19:40:54Z,...,False,False,Belinda Carlisle,belinda carlisle,heaven is a place on earth,NaN,NaN,NaN,NaN,NaN


## 7. Build a missing-tracks shopping list
Highlight Spotify tracks that are still missing from `Music/` so you can go hunt them down.

In [43]:
def build_shopping_list(matched_df: pd.DataFrame) -> pd.DataFrame:
    if matched_df.empty:
        return pd.DataFrame()
    missing = matched_df[matched_df['file_path'].isna()].copy()
    if missing.empty:
        return pd.DataFrame()

    grouped = (
        missing.groupby(['artist_canonical', 'title_canonical'], as_index=False)
        .agg({
            'primary_artist': 'first',
            'Track Name': 'first',
            'Album Name': lambda col: col.dropna().iloc[0] if col.dropna().any() else pd.NA,
            'Duration (ms)': 'first',
            'playlist_name': lambda col: sorted(set(col)),
            'is_liked': 'any',
            'is_top_songs': 'any'
        })
    )
    grouped['Playlists_Count'] = grouped['playlist_name'].apply(len)
    grouped['Playlists'] = grouped['playlist_name'].apply(lambda names: '; '.join(names))
    grouped.rename(columns={
        'primary_artist': 'Artist',
        'Track Name': 'Title',
        'Album Name': 'Album',
        'Duration (ms)': 'Duration_ms',
        'is_liked': 'Is_Liked',
        'is_top_songs': 'Is_Top_Songs'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'Playlists_Count', 'Playlists', 'Is_Liked', 'Is_Top_Songs']
    grouped = grouped[columns]
    grouped.sort_values(['Playlists_Count', 'Is_Liked', 'Artist', 'Title'], ascending=[False, False, True, True], inplace=True)
    return grouped

shopping_df = build_shopping_list(matched_df)
if shopping_df.empty:
    print('All playlist tracks already exist locally – no shopping list generated.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    shopping_path = SHOPPING_LIST_DIR / f'shopping_list_{timestamp}.csv'
    shopping_df.to_csv(shopping_path, index=False)
    print(f"Shopping list saved to {shopping_path}")
    display(shopping_df.head())

Shopping list saved to c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists\shopping_list_2025-11-18-17-54-19.csv


,Artist,Title,Album,Duration_ms,Playlists_Count,Playlists,Is_Liked,Is_Top_Songs
5305,Point Break Candy,Hole In The Sun (feat. COS & Conway) [From Cyb...,Hole In The Sun (feat. COS & Conway) [From Cyb...,208066.0,11,2020s Dance; Cyberpunk 2077; Cyberpunk 2077 Ra...,True,True
5788,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),156943.0,11,2020s Dance; Favorites; Liked Songs; Mega Hit ...,True,True
3989,Lil Nas X,INDUSTRY BABY,INDUSTRY BABY 2.0,212000.0,10,Favorites; Liked Songs; New Favorites; Summer ...,True,True
350,Apashe,Uebok Gotta Run,Renaissance,163500.0,9,Favorites; Hyper Glitch & Happy Hardcore; Like...,True,True
385,Ariana Grande,the light is coming (feat. Nicki Minaj),Sweetener,228373.0,8,2020s Dance; Favorites; Liked Songs; New Favor...,True,True


## 8. Generate an orphaned-tracks list
Highlight tracks that exist in `Music/` but are not referenced by any current playlist snapshot.

In [44]:
def build_orphaned_tracks(matched_df: pd.DataFrame, library_df: pd.DataFrame) -> pd.DataFrame:
    if library_df.empty:
        return pd.DataFrame()
    playlist_keys = set(zip(matched_df['artist_canonical'], matched_df['title_canonical'])) if not matched_df.empty else set()
    library_df = library_df.copy()
    library_df['key'] = list(zip(library_df['artist_canonical'], library_df['title_canonical']))
    mask = library_df['key'].apply(lambda key: key not in playlist_keys)
    orphaned = library_df[mask].copy()
    if orphaned.empty:
        return pd.DataFrame()
    orphaned.rename(columns={
        'artist_raw': 'Artist',
        'title_raw': 'Title',
        'album_raw': 'Album',
        'duration_ms': 'Duration_ms'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'file_path']
    return orphaned[columns]

orphan_df = build_orphaned_tracks(matched_df, library_index)
if orphan_df.empty:
    print('No orphaned tracks – every local track appears in at least one playlist snapshot.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    orphan_path = SHOPPING_LIST_DIR / f'orphaned_tracks_{timestamp}.csv'
    orphan_df.to_csv(orphan_path, index=False)
    print(f"Orphaned-track report saved to {orphan_path}")
    display(orphan_df.head())

Orphaned-track report saved to c:\Users\CJ\Documents\GitHub\Exodusify\shopping_lists\orphaned_tracks_2025-11-18-17-54-19.csv


,Artist,Title,Album,Duration_ms,file_path
17,6EJOU,I'm Not Your Mate,I'm Not Your Mate,316761,6EJOU/I'm Not Your Mate/01 I'm Not Your Mate.mp3
19,"753, Brecc",Dare - Brecc Remix,Dare EP,366367,"753, Brecc/Dare EP/03 Dare - Brecc Remix.mp3"
20,"A Boogie Wit da Hoodie, Kodak Black",Drowning (feat. Kodak Black),Drowning (feat. Kodak Black),209319,"A Boogie Wit da Hoodie, Kodak Black/Drowning (..."
52,"A Perfect Circle, James Iha",Blue - Bird Shake Mix,Complete Collection,237166,"A Perfect Circle, James Iha/Complete Collectio..."
51,"A Perfect Circle, James Iha",Outsider - Frosted Yogurt Mix,aMOTION,247562,"A Perfect Circle, James Iha/aMOTION/05 Outside..."


## 9. Show Playlist Statistics
Summarize key statistics about each playlist, including total tracks, matched tracks, missing tracks, and orphaned tracks.

In [45]:
if 'matched_df' not in globals():
    print("Run cells 1-7 to create 'matched_df' before generating playlist stats.")
elif matched_df.empty:
    print('Playlist DataFrame is empty – load Spotify CSVs first.')
else:
    stats = (
        matched_df
        .groupby('playlist_name', dropna=False)
        .agg(
            Total_Tracks=('Track Name', 'size'),
            Matched_Tracks=('file_path', lambda col: col.notna().sum()),
            Liked_Snapshot=('is_liked', 'any'),
            Top_Songs_Snapshot=('is_top_songs', 'any')
        )
        .reset_index()
    )
    stats['Missing_Tracks'] = stats['Total_Tracks'] - stats['Matched_Tracks']
    stats['Percent_Complete'] = (stats['Matched_Tracks'] / stats['Total_Tracks'] * 100).round(1)
    stats.sort_values(['Percent_Complete', 'playlist_name'], ascending=[False, True], inplace=True)

    overall_total = int(stats['Total_Tracks'].sum())
    overall_missing = int(stats['Missing_Tracks'].sum())
    overall_matched = overall_total - overall_missing

    missing_unique = (
        matched_df[matched_df['file_path'].isna()]
        .drop_duplicates(subset=['artist_canonical', 'title_canonical'])
        .shape[0]
    )

    print(
        f"Playlists analyzed: {len(stats)} | Tracks: {overall_total:,} | "
        f"Matched: {overall_matched:,} | Missing: {overall_missing:,}"
    )
    print(f"Unique missing tracks across all playlists: {missing_unique:,}")
    display(stats)


Playlists analyzed: 74 | Tracks: 15,005 | Matched: 4,420 | Missing: 10,585
Unique missing tracks across all playlists: 7,426


,playlist_name,Total_Tracks,Matched_Tracks,Liked_Snapshot,Top_Songs_Snapshot,Missing_Tracks,Percent_Complete
19,Get Pumped,1,1,False,False,0,100.0
56,Torque - Early,9,7,False,False,2,77.8
16,Florida,13,10,False,False,3,76.9
62,Your Top Songs 2023,104,79,False,True,25,76.0
35,Night drive,4,3,False,False,1,75.0
...,...,...,...,...,...,...,...
49,The Mandalorian Soundtrack (Disney+),119,0,False,False,119,0.0
53,Today’s Top Hits,50,0,False,False,50,0.0
67,planet rave,160,0,False,False,160,0.0
70,vaporwave,100,0,False,False,100,0.0


## 3. Process new additions
Move fresh MP3 downloads from the staging `Add/` folder into the canonical `Music/Artist/Album/Title.mp3` structure before scanning the library.

In [46]:
def process_new_additions(add_root: Path, music_root: Path, playlist_csv_root: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Move staged downloads from Add/ into Music/ and summarize the results."""
    SUPPORTED_IMPORT_SUFFIXES = {'.mp3', '.flac', '.m4a', '.aac', '.ogg', '.wav'}
    PLAYLIST_STAGE_FOLDER = "To Playlist"

    add_root.mkdir(exist_ok=True)
    playlist_staging_root = add_root / PLAYLIST_STAGE_FOLDER
    playlist_staging_root.mkdir(parents=True, exist_ok=True)

    # Ensure playlist staging folders mirror the Spotify CSV exports for easy triage.
    staging_lookup: dict[Path, str] = {}
    playlist_csvs = sorted(playlist_csv_root.glob('*.csv')) if playlist_csv_root.exists() else []
    for csv_file in playlist_csvs:
        playlist_name = friendly_playlist_name(csv_file)
        staging_dir = playlist_staging_root / safe_path_component(playlist_name, 'Playlist')
        staging_lookup[staging_dir] = playlist_name
        staging_dir.mkdir(parents=True, exist_ok=True)
    if playlist_csvs:
        print(
            "Ensured playlist staging folders exist for "
            f"{len(playlist_csvs)} playlists under {playlist_staging_root}."
        )
    else:
        print(
            f"No Spotify CSVs detected in {playlist_csv_root} – add files under Add/ manually or export playlists first."
        )

    actions: list[dict[str, object]] = []
    add_files = sorted(add_root.rglob('*'))
    for file_path in add_files:
        if not file_path.is_file():
            continue
        if file_path.suffix.lower() not in SUPPORTED_IMPORT_SUFFIXES:
            continue
        action: dict[str, object] = {
            'source': file_path.relative_to(add_root).as_posix(),
            'destination': pd.NA,
            'playlist_target': pd.NA,
            'status': None,
            'reason': pd.NA,
            'artist': pd.NA,
            'album': pd.NA,
            'title': pd.NA,
        }

        if not file_path.exists():
            action['status'] = 'skipped_missing_source'
            action['reason'] = 'File disappeared before processing.'
            actions.append(action)
            continue

        try:
            rel_to_staging = file_path.relative_to(playlist_staging_root)
        except ValueError:
            rel_to_staging = None
        if rel_to_staging:
            rel_stage_parts = rel_to_staging.parts
            if rel_stage_parts:
                staging_dir = playlist_staging_root / rel_stage_parts[0]
                if staging_dir in staging_lookup:
                    action['playlist_target'] = staging_lookup[staging_dir]

        try:
            audio = MutagenFile(file_path)
        except Exception as exc:
            action['status'] = 'error_read'
            action['reason'] = str(exc)
            actions.append(action)
            continue

        tags = getattr(audio, 'tags', None) if audio else None
        artist_tag = tags.get('TPE1') or tags.get('artist') if tags else None
        title_tag = tags.get('TIT2') or tags.get('title') if tags else None
        album_tag = tags.get('TALB') or tags.get('album') if tags else None

        def tag_value(raw_tag) -> Optional[str]:
            if hasattr(raw_tag, 'text') and raw_tag.text:
                return str(raw_tag.text[0]).strip()
            if isinstance(raw_tag, str):
                return raw_tag.strip()
            return None

        artist_value = tag_value(artist_tag)
        album_value = tag_value(album_tag)
        title_value = tag_value(title_tag)

        rel_parent_parts = list(file_path.relative_to(add_root).parts)
        if rel_parent_parts and rel_parent_parts[0] == PLAYLIST_STAGE_FOLDER:
            rel_parent_parts = rel_parent_parts[1:]
        if not artist_value and rel_parent_parts:
            artist_value = rel_parent_parts[0]
        if not album_value and len(rel_parent_parts) > 1:
            album_value = rel_parent_parts[1]
        if not title_value:
            title_value = file_path.stem

        if not artist_value:
            action['status'] = 'skipped_unknown_artist'
            action['reason'] = 'Missing artist tag after fallbacks.'
            actions.append(action)
            continue
        if not album_value:
            action['status'] = 'skipped_unknown_album'
            action['reason'] = 'Missing album tag after fallbacks.'
            actions.append(action)
            continue
        if not title_value:
            action['status'] = 'skipped_missing_tags'
            action['reason'] = 'Unable to infer track title.'
            actions.append(action)
            continue

        artist_component = safe_path_component(artist_value, 'Unknown Artist')
        album_component = safe_path_component(album_value, 'Unknown Album')
        title_component = safe_path_component(title_value, file_path.stem)
        dest_dir = music_root / artist_component / album_component
        dest_path = dest_dir / f"{title_component}{file_path.suffix.lower()}"

        canonical_title = canonicalize_string(title_value)
        duplicate_target = None
        if dest_dir.exists():
            for existing_file in dest_dir.iterdir():
                if not existing_file.is_file():
                    continue
                existing_canonical = canonicalize_string(existing_file.stem)
                if existing_canonical == canonical_title:
                    duplicate_target = existing_file
                    break

        if dest_path.exists():
            action['status'] = 'skipped_exists'
            action['reason'] = f"Destination already exists: {dest_path}"
            actions.append(action)
            continue
        if duplicate_target is not None:
            action['status'] = 'skipped_duplicate_title'
            action['reason'] = f"Similar track already present: {duplicate_target}"
            actions.append(action)
            continue

        try:
            dest_dir.mkdir(parents=True, exist_ok=True)
            shutil.move(str(file_path), str(dest_path))
        except Exception as exc:
            action['status'] = 'error_move'
            action['reason'] = str(exc)
            actions.append(action)
            continue

        action['status'] = 'moved'
        action['destination'] = dest_path.relative_to(music_root).as_posix()
        action['artist'] = artist_value
        action['album'] = album_value
        action['title'] = title_value
        actions.append(action)

    result_df = pd.DataFrame(actions)

    playlist_updates_df = pd.DataFrame()
    if not result_df.empty:
        moved = result_df[result_df['status'] == 'moved']
        if not moved.empty:
            playlist_updates_df = (
                moved.groupby('playlist_target', dropna=True)
                .size()
                .reset_index(name='tracks_added')
                .rename(columns={'playlist_target': 'playlist_name'})
            )
            playlist_updates_df.sort_values('tracks_added', ascending=False, inplace=True)

    return result_df, playlist_updates_df


new_additions_log, playlist_updates_log = process_new_additions(ADD_ROOT, MUSIC_ROOT, SPOTIFY_PLAYLISTS)
if not new_additions_log.empty:
    display(new_additions_log)
if playlist_updates_log is not None and not playlist_updates_log.empty:
    print('Updated playlists based on Add/ staging folders:')
    display(playlist_updates_log)

Ensured playlist staging folders exist for 79 playlists under c:\Users\CJ\Documents\GitHub\Exodusify\Add\To Playlist.
